In [5]:
import datetime
import hashlib
import json
from flask import Flask, jsonify
import pandas as pd

In [27]:
class Blockchain:

    def __init__(self):
        self.chain = []
        self.validators = ['validator1', 'validator2']  # Example validators
        self.create_block(proof=1, previous_hash='0')#, validator='genesis_validator')
        self.supply_chain_contract = self.SupplyChainSmartContract()
        # self.escrow = self.escrow(platform_arbiter="PlatformArbiter")
        # self.wallets = {"Alice": self.Wallet("Alice", 500),
        #                 "Bob": self.Wallet("Bob", 500),
        #                 "PlatformArbiter": self.Wallet("PlatformArbiter", 500)}
    # ------------------------------------------------------------------------------------------------
    def create_block(self, proof, previous_hash):#, validator):
        # if validator not in self.validators:
        #     return "Validator is not authorized to create block", 403
        
        block = {'index': len(self.chain) + 1,
                 'timestamp': str(datetime.datetime.now()),
                 'proof': proof,
                 'previous_hash': previous_hash}
        self.chain.append(block)
        return block
    # ------------------------------------------------------------------------------------------------
    def print_previous_block(self):
        if self.chain:
            return self.chain[-1]
        else:
            return None
    # ------------------------------------------------------------------------------------------------
    def proof_of_work(self, previous_proof):
        new_proof = 1
        check_proof = False
        while check_proof is False:
            hash_operation = hashlib.sha256(
                str(new_proof**2 - previous_proof**2).encode()).hexdigest()
            if hash_operation[:5] == '00000':
                check_proof = True
            else:
                new_proof += 1
        return new_proof
    # ------------------------------------------------------------------------------------------------
    def hash_(self, block):
        encoded_block = json.dumps(block, sort_keys=True).encode()
        return hashlib.sha256(encoded_block).hexdigest()
    # ------------------------------------------------------------------------------------------------
    def chain_valid(self, chain):
        previous_block = chain[0]
        block_index = 1
 
        while block_index < len(chain):
            block = chain[block_index]
            if block['previous_hash'] != self.hash_(previous_block):
                return False
 
            previous_proof = previous_block['proof']
            proof = block['proof']
            hash_operation = hashlib.sha256(
                str(proof**2 - previous_proof**2).encode()).hexdigest()
 
            if hash_operation[:5] != '00000':
                return False
            previous_block = block
            block_index += 1
 
        return True

    # ------------------------------------------------------------------------------------------------
    class SupplyChainSmartContract:
        def __init__(self):
            self.orders = {}
            self.order_status = {}

        def place_order(self, customer_id, retailer_id, product_id):
            order_id = len(self.orders) + 1
            self.orders[order_id] = {'customer_id': customer_id,
                                     'retailer_id': retailer_id,
                                     'product_id': product_id,
                                     'status': 'placed'}
            print(f"Order {order_id} placed for product {product_id} by customer {customer_id} through retailer {retailer_id}.")
            return order_id

        def fulfill_order(self, order_id, supplier_warehouse_id, role):
            if order_id in self.orders and self.orders[order_id]['status'] == 'placed':
                self.orders[order_id]['supplier_warehouse_id'] = supplier_warehouse_id
                self.orders[order_id]['status'] = 'shipped'
                print(f"Order {order_id} shipped by {role} {supplier_warehouse_id}.")
            else:
                print(f"Order {order_id} cannot be fulfilled by {role} {supplier_warehouse_id}.")

        def confirm_receipt(self, order_id, customer_id):
            if order_id in self.orders and self.orders[order_id]['customer_id'] == customer_id and self.orders[order_id]['status'] == 'shipped':
                self.orders[order_id]['status'] = 'received'
                print(f"Customer {customer_id} confirmed receipt of order {order_id}.")
            else:
                print(f"Order {order_id} receipt cannot be confirmed by customer {customer_id}.")

        def verify_order(self, order_id, retailer_id):
            if order_id in self.orders and self.orders[order_id]['retailer_id'] == retailer_id and self.orders[order_id]['status'] == 'received':
                print(f"Order {order_id} completion verified by retailer {retailer_id}.")
                return True
            print(f"Order {order_id} cannot be verified by retailer {retailer_id}.")
            return False
        
app = Flask(__name__)
blockchain = Blockchain()

# block mining
@app.route('/mine_block', methods=['GET'])
def mine_block():
    previous_block = blockchain.print_previous_block()
    previous_proof = previous_block['proof']
    proof = blockchain.proof_of_work(previous_proof)
    previous_hash = blockchain.hash(previous_block)
    block = blockchain.create_block(proof, previous_hash)
 
    response = {'message': 'A block is MINED',
                'index': block['index'],
                'timestamp': block['timestamp'],
                'proof': block['proof'],
                'previous_hash': block['previous_hash']}
 
    return jsonify(response), 200

# display chain
@app.route('/get_chain', methods=['GET'])
def display_chain():
    response = {'chain': blockchain.chain,
                'length': len(blockchain.chain)}
    return jsonify(response), 200

# valid
@app.route('/valid', methods=['GET'])
def valid():
    valid = blockchain.chain_valid(blockchain.chain)
    if valid:
        response = {'message': 'The Blockchain is valid.'}
    else:
        response = {'message': 'The Blockchain is not valid.'}
    return jsonify(response), 200

def supplychainSC(customer_id, retailer_id, product_id=None, supplier_warehouse_id=None, role=None):
    # Use the instance of SupplyChainSmartContract to call its methods
    if product_id is not None:
        order_id = blockchain.supply_chain_contract.place_order(customer_id, retailer_id, product_id)
    else:
        return "Product ID is required to place an order", None, None
    
    if supplier_warehouse_id is not None and role is not None:
        fulfill = blockchain.supply_chain_contract.fulfill_order(order_id, supplier_warehouse_id, role)
    else:
        fulfill = "Order cannot be fulfilled without supplier warehouse ID and role"
    
    if order_id:
        receipt = blockchain.supply_chain_contract.confirm_receipt(order_id, customer_id)
        verify = blockchain.supply_chain_contract.verify_order(order_id, retailer_id)
    else:
        receipt = "No order to confirm receipt for"
        verify = "No order to verify"
    
    return fulfill, receipt, verify

# supplychainSC(1,121,54,23,'Supplier')

In [28]:
supplychainSC(customer_id=1,retailer_id=121,product_id=54)#,supplier_warehouse_id=23,role='Supplier')

Order 1 placed for product 54 by customer 1 through retailer 121.
Order 1 receipt cannot be confirmed by customer 1.
Order 1 cannot be verified by retailer 121.


('Order cannot be fulfilled without supplier warehouse ID and role',
 None,
 False)

In [141]:
#     # ------------------------------------------------------------------------------------------------
#     class Wallet:
#         transactions = []  # Class variable to store all transactions
#         def __init__(self, owner, balance):
#             self.owner = owner
#             self.balance = balance
#         # ------------------------------------------------------------------------------------------------    
#         def deposit(self, amount):
#             self.balance += amount
#             Blockchain.Wallet.transactions.append({"owner": self.owner, 
#                                                    "type": "deposit", 
#                                                    "amount": amount, 
#                                                    "balance": self.balance})
#         # ------------------------------------------------------------------------------------------------   
#         def withdraw(self, amount):
#             if amount > self.balance:
#                 raise Exception("Insufficient balance.")
#             self.balance -= amount
#             Blockchain.Wallet.transactions.append({"owner": self.owner, 
#                                                    "type": "withdraw", 
#                                                    "amount": -amount, 
#                                                    "balance": self.balance})
#        # ------------------------------------------------------------------------------------------------     
#         def get_balance(self):
#             return self.balance
#        # ------------------------------------------------------------------------------------------------     
#         def get_transaction(self):
#             return self.transaction
#     # ------------------------------------------------------------------------------------------------
#     def record_transaction(self, transaction_type, from_, to, amount, product_id=None):
#         if from_ in self.wallets and to in self.wallets:
#             from_wallet = self.wallets[from_]
#             to_wallet = self.wallets[to]

#             if transaction_type == 'deposit':
#                 to_wallet.deposit(amount)
#             elif transaction_type == 'withdraw':
#                 from_wallet.withdraw(amount)
#             elif transaction_type == 'transfer' or transaction_type == 'purchase':
#                 from_wallet.withdraw(amount)
#                 to_wallet.deposit(amount)

#             # Record the transaction with updated balances
#             new_transaction = {
#                 'timestamp': datetime.datetime.now(),
#                 'transaction_type': transaction_type,
#                 'from': from_, 
#                 'sender wallet': from_wallet.get_balance(), 
#                 'to': to, 
#                 'receiver wallet': to_wallet.get_balance(), 
#                 'amount': amount,
#                 'product_id': product_id
#             }

#             if not hasattr(self, 'transactions_df'):
#                 self.transactions_df = pd.DataFrame(columns=['timestamp', 'transaction_type', 'from', 'sender wallet', 'to', 'receiver wallet', 'amount'])

#             self.transactions_df = self.transactions_df.append(new_transaction, ignore_index=True)
#         else:
#             print("Error: One or both of the wallet participants do not exist.")
#     # ------------------------------------------------------------------------------------------------
#     class escrow:
#         def __init__(self, platform_arbiter):
#             self.transactions = {}
#             self.platform_arbiter = platform_arbiter
#         # ------------------------------------------------------------------------------------------------
#         def initiate_transaction(self, product_id, seller, buyer, amount):
#             if product_id in self.transactions:
#                 raise ValueError("Transaction already initiated for this product.")
#             self.transactions[product_id] = {
#                 "seller": seller,
#                 "buyer": buyer,
#                 "amount": amount,
#                 "deposited_amount": 0,
#                 "is_item_received": False,
#                 "status": "awaiting_payment"
#             }
#             print(f"Transaction for product {product_id} initiated.")
#         # ------------------------------------------------------------------------------------------------
#         def pay_to_escrow(self, product_id, amount):
#             transaction = self.transactions.get(product_id)
#             if not transaction:
#                 raise ValueError("Transaction does not exist.")
#             if amount != transaction["amount"]:
#                 raise ValueError("Payment amount must match the agreed amount.")
#             transaction["deposited_amount"] = amount
#             transaction["status"] = "awaiting_shipment"
#             print(f"{amount} paid to escrow for product {product_id}.")
#         # ------------------------------------------------------------------------------------------------
#         def confirm_receipt(self, product_id, buyer):
#             transaction = self.transactions.get(product_id)
#             if not transaction:
#                 raise ValueError("Transaction does not exist.")
#             if transaction["buyer"] != buyer:
#                 raise ValueError("Only the buyer can confirm receipt.")
#             transaction["is_item_received"] = True
#             transaction["status"] = "completed"
#             print(f"Receipt of product {product_id} confirmed.")
#         # ------------------------------------------------------------------------------------------------
#         def release_funds_to_seller(self, product_id):
#             transaction = self.transactions.get(product_id)
#             if not transaction:
#                 raise ValueError("Transaction does not exist.")
#             if not transaction["is_item_received"]:
#                 raise ValueError("Item has not been received yet. Funds cannot be released.")
#             print(f"Funds released to {transaction['seller']} for product {product_id}.")
#             transaction["deposited_amount"] = 0  # Clear the escrow
#     # ------------------------------------------------------------------------------------------------
#     def get_transactions_df(self):
#         """Return the transactions DataFrame."""
#         if hasattr(self, 'transactions_df'):
#             return self.transactions_df
#         else:
#             return pd.DataFrame()  # Return an empty DataFrame if no transactions have been recorded
        
app = Flask(__name__)
blockchain = Blockchain()

# block mining
@app.route('/mine_block', methods=['GET'])
def mine_block():
    previous_block = blockchain.print_previous_block()
    previous_proof = previous_block['proof']
    proof = blockchain.proof_of_work(previous_proof)
    previous_hash = blockchain.hash(previous_block)
    block = blockchain.create_block(proof, previous_hash)
 
    response = {'message': 'A block is MINED',
                'index': block['index'],
                'timestamp': block['timestamp'],
                'proof': block['proof'],
                'previous_hash': block['previous_hash']}
 
    return jsonify(response), 200

# display chain
@app.route('/get_chain', methods=['GET'])
def display_chain():
    response = {'chain': blockchain.chain,
                'length': len(blockchain.chain)}
    return jsonify(response), 200

# valid
@app.route('/valid', methods=['GET'])
def valid():
    valid = blockchain.chain_valid(blockchain.chain)
    if valid:
        response = {'message': 'The Blockchain is valid.'}
    else:
        response = {'message': 'The Blockchain is not valid.'}
    return jsonify(response), 200

# record transaction
# blockchain.record_transaction('transfer',from_='Alice',to='Bob',amount= 100)
# blockchain.record_transaction('purchase', product_id= '12345', to= 'PlatformArbiter', from_= 'Alice', amount= 100)
# blockchain.get_transactions_df()

In [ ]:
blockchain = Blockchain()  # Assuming you've properly initialized Blockchain earlier

# Example of deposit to show transactions being added
alice_wallet = blockchain.Wallet("Alice", 500)
alice_wallet.deposit(100)

# Retrieve and print all transactions
all_transactions = blockchain.Wallet.get_transaction()
print(all_transactions)

In [84]:
# ------------------------------------------------------------------------------------------------
class Wallet:
    transactions = []  # Class variable to store all transactions
    def __init__(self, owner, balance):
        self.owner = owner
        self.balance = balance
    # ------------------------------------------------------------------------------------------------    
    def deposit(self, amount):
        self.balance += amount
        Blockchain.Wallet.transactions.append({"owner": self.owner, 
                                               "type": "deposit", 
                                               "balance": self.balance,
                                               "amount": amount,
                                               "to":})
    # ------------------------------------------------------------------------------------------------   
    def withdraw(self, amount):
        if amount > self.balance:
            raise Exception("Insufficient balance.")
        self.balance -= amount
        Blockchain.Wallet.transactions.append({"owner": self.owner, 
                                               "type": "withdraw", 
                                               "amount": -amount, 
                                               "balance": self.balance})
   # ------------------------------------------------------------------------------------------------     
    def get_balance(self):
        return self.balance
   # ------------------------------------------------------------------------------------------------     
    def get_transaction(self):
        return self.transaction
# ------------------------------------------------------------------------------------------------  
def record_transaction(self, transaction_type, from_, to, amount, product_id=None):
        if from_ in self.wallets and to in self.wallets:
            from_wallet = self.wallets[from_]
            to_wallet = self.wallets[to]

            if transaction_type == 'deposit':
                to_wallet.deposit(amount)
            elif transaction_type == 'withdraw':
                from_wallet.withdraw(amount)
            elif transaction_type == 'transfer' or transaction_type == 'purchase':
                from_wallet.withdraw(amount)
                to_wallet.deposit(amount)

            # Record the transaction with updated balances
            new_transaction = {
                'timestamp': datetime.datetime.now(),
                'transaction_type': transaction_type,
                'from': from_, 
                'sender wallet': from_wallet.get_balance(), 
                'to': to, 
                'receiver wallet': to_wallet.get_balance(), 
                'amount': amount,
                'product_id': product_id
            }

            if not hasattr(self, 'transactions_df'):
                self.transactions_df = pd.DataFrame(columns=['timestamp', 'transaction_type', 'from', 'sender wallet', 'to', 'receiver wallet', 'amount'])

            self.transactions_df = self.transactions_df.append(new_transaction, ignore_index=True)
        else:
            print("Error: One or both of the wallet participants do not exist.")